In [ ]:
%pip install scanpy scvi-tools anndata
%pip install --quiet gdown
%pip install seaborn
%pip install --quiet bbknn
%pip install pandas
%pip install pyscenic
%pip install ipywidgets
%pip install MulticoreTSNE
%pip install scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached MulticoreTSNE-0.1.tar.gz (20 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for MulticoreTSNE
Failed to build MulticoreTSNE
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [26 lines of output]
      c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\setuptools\_distutils\dist.py:261: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\setuptools\_distutils\dist.py:261: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\MulticoreTSNE
      copying MulticoreTSNE\__init__.py -> build\lib.win-amd64-cpython-312\MulticoreTSNE
      creating build\lib.win-amd64-cpython-312\MulticoreTSNE\tests
      copying MulticoreTSNE\tests\test_base.py -> build\lib.win-amd64-cpython-312\MulticoreTSNE\tests
      copying MulticoreTSNE\tests\__init__.py -> bui

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement scipy.io (from versions: none)
ERROR: No matching distribution found for scipy.io


Import packages

In [33]:
import scanpy as sc
import anndata as ad
import os
import tarfile
import scvi
import numpy as np
import glob

#specific requirements of pyscenic
import loompy as lp
## This is T-SNE, download has wheel error, could fix... could use UM
#from MulticoreTSNE import MulticoreTSNE as TSNE

#data vis
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import umap

In [65]:
import shutil

total, used, free = shutil.disk_usage("/")
print(f"Total: {total // (2**30)} GiB")
print(f"Used: {used // (2**30)} GiB")
print(f"Free: {free // (2**30)} GiB")

Total: 475 GiB
Used: 441 GiB
Free: 34 GiB


In [77]:
import scanpy as sc
import anndata

adata_example = sc.read_10x_mtx("FRZFRESH_GX26_ES345")
adata_example

FileNotFoundError: Did not find file FRZFRESH_GX26_ES345\matrix.mtx.gz.

In [80]:
import os
import tarfile

# Function to extract tar.gz files
def extract_tar_gz(tar_gz_path, extract_dir):
    with tarfile.open(tar_gz_path, "r:gz") as tar:
        tar.extractall(path=extract_dir)

# Directory containing your datasets
input_dir = "input/biostudiesarrayE-MTAB-14039"
extracted_dir = "input/extracted_data"  # Folder to save the extracted files
os.makedirs(extracted_dir, exist_ok=True)

# List all top-level files in the input directory
tar_gz_files = [f for f in os.listdir(input_dir) if f.endswith('.tar.gz')]

# Extract tar.gz files
for tar_gz_file in tar_gz_files:
    tar_gz_path = os.path.join(input_dir, tar_gz_file)
    
    # Extract the tar.gz file to the extracted directory
    extract_tar_gz(tar_gz_path, extracted_dir)


In [94]:
import os
import scanpy as sc
# Directory where the files have been extracted
extracted_work_dir = "input/extracted_data"  # Directory where you extracted the tar files

# List to store directory names under the 'work' directory
dataset_dirs = []

# Traverse the 'work' directory to find subdirectories
for root, dirs, files in os.walk(extracted_work_dir):
    # Only consider directories directly under the 'work' directory
    if "work" in root:
        for dir_name in dirs:
            
            # Append the directory name under 'work'
            dataset_dirs.append(path)

# Now dataset_dirs contains the names of all subdirectories under 'work'
print("Dataset directories under 'work':", dataset_dirs)


# List to store AnnData objects for merging
adatas = []
batch_size = 5  # Adjust based on available memory
for i in range(0, len(dataset_dirs), batch_size):
    # Get the current batch of files
    batch_files = dataset_dirs[i:i+batch_size]
    
    # Load the current batch of datasets
    batch_adatas = []
    for mtx_file in batch_files:
        print(mtx_file)
        # Assuming each batch is loaded from corresponding directories
        adata = sc.read_10x_mtx(mtx_file)
        batch_adatas.append(adata)
    
    # Concatenate the current batch
    if adatas:
        adatas[0] = anndata.concat([adatas[0]] + batch_adatas)
        adatas[0].obs_names_make_unique() 
    else:
        adatas = batch_adatas

    # Free memory by clearing the batch Adatas
    batch_adatas = []

Dataset directories under 'work': ['input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX9006', 'input/extracted_data/work/UA_Endo9426402_FX900

c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\anndata\_core\anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006


c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\anndata\_core\anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\anndata\utils.py:261: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['AAAGTGAGTTTACACG-1-1', 'AAAGTGAGTTTACACG-1-2', 'AAAGTGAGTTTACACG-1-3', 'AAAGTGAGTTTACACG-1-4', 'AAAGTGAGTTTACACG-1-5']
  warnings.warn(


input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006


...

c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\anndata\_core\anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\anndata\utils.py:261: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['AAAGTGAGTTTACACG-1-1', 'AAAGTGAGTTTACACG-1-2', 'AAAGTGAGTTTACACG-1-3', 'AAAGTGAGTTTACACG-1-4', 'AAAGTGAGTTTACACG-1-5']
  warnings.warn(


input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006


c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\anndata\_core\anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\anndata\utils.py:261: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['AAAGTGAGTTTACACG-1-1', 'AAAGTGAGTTTACACG-1-2', 'AAAGTGAGTTTACACG-1-3', 'AAAGTGAGTTTACACG-1-4', 'AAAGTGAGTTTACACG-1-5']
  warnings.warn(


input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006
input/extracted_data/work/UA_Endo9426402_FX9006


c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\anndata\_core\anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\crisi\Desktop\Code\UW\CompBio\endometrial_analysis\.venv\Lib\site-packages\anndata\utils.py:261: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['AAAGTGAGTTTACACG-1-1', 'AAAGTGAGTTTACACG-1-2', 'AAAGTGAGTTTACACG-1-3', 'AAAGTGAGTTTACACG-1-4', 'AAAGTGAGTTTACACG-1-5']
  warnings.warn(


Load in data -- OLAF's VERSION. Garrett Depending on what you did you made need to make a different block.

Data Exploration

In [95]:
bdata = adatas[0].copy()
bdata.obs.head()


""
AAAGTGAGTTTACACG-1
AACAACCTCGTAGCCG-1
AACCATGGTAAGATTG-1
AACCTTTTCTAGACCA-1
AACGGGACAGCAATTC-1


In [96]:


bdata.var.head()



""
MIR1302-2HG
FAM138A
OR4F5
AL627309.1
AL627309.3


In [99]:
print(bdata.var_names)
bdata.X

Index(['MIR1302-2HG', 'FAM138A', 'OR4F5', 'AL627309.1', 'AL627309.3',
       'AL627309.2', 'AL627309.5', 'AL627309.4', 'AP006222.2', 'AL732372.1',
       ...
       'AC133551.1', 'AC136612.1', 'AC136616.1', 'AC136616.3', 'AC136616.2',
       'AC141272.1', 'AC023491.2', 'AC007325.1', 'AC007325.4', 'AC007325.2'],
      dtype='object', length=36601)


<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 99304128 stored elements and shape (15642, 36601)>

In [102]:
bdata.obs
output_file = "output/integrated_data.h5ad"
bdata.write(output_file)

OSError: [Errno 28] Driver write request failed (file write failed: time = Thu Nov  7 03:26:38 2024
, filename = 'output\integrated_data.h5ad', file descriptor = 3, errno = 28, error message = 'No space left on device', buf = 0000023CF79A7608, total write size = 96980, bytes this sub-write = 96980, bytes actually written = 18446744073709551615, offset = 117982728)

## Quality Control

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)


adata.obs.head()



In [ ]:
sc.pl.violin(
    adata, 
    [
     'n_genes_by_counts', 
     'total_counts', 
     'pct_counts_mt'
     ],
    multi_panel=True
)

In [ ]:
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts")

### Normalization

In [ ]:
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4) 
sc.pp.log1p(adata) 
adata.raw = adata

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=4000,
    # subset=True, # to automatically subset to the 4000 genes
    layer="counts",
    flavor="seurat_v3"
)
sc.pl.highly_variable_genes(adata, log=True)

In [ ]:
adata = adata[:, adata.var.highly_variable].copy()
adata.raw.to_adata()

### Standard Workflow

In [ ]:
sc.pp.scale(adata) # z normalize the columns (genes)
sc.tl.pca(adata)


adata.obsm["X_pca"]
sc.pp.neighbors(adata) # compute nearest neighbors
sc.tl.umap(adata)
sc.pl.umap(adata, color="batch")

IF THE CLUSTERS ARE BY BATCH:: ADD THIS CODE:

In [ ]:

import scanpy.external as sce
sce.pp.bbknn(adata, batch_key="batch")
sc.tl.umap(adata)
sc.pl.umap(adata, color=["batch", "cell_types"], ncols=1)


IF IT STILL DOESN'T WORK THEN REDO THE LOAD IN AND USE SCIVI
https://ccbskillssem.github.io/assets/scvi_notebook.html

https://www.youtube.com/watch?v=EKTg9NV5hEA

## SCENIC

##### STEP 1: IDENTIFY TF with arboreto 


In [ ]:
f_tfs = "/ddn1/vol1/staging/leuven/stg_00002/lcb/cflerin/resources/allTFs_hg38.txt"

!pyscenic grn {f_loom_path_scenic} {f_tfs} -o adj.csv --num_workers 20

adjacencies = pd.read_csv("adj.tsv", index_col=False, sep='\t')
adjacencies.head()

##### STEP 2: Refine regulons through pruning 

In [ ]:

# ranking databases
f_db_glob = "/ddn1/vol1/staging/leuven/res_00001/databases/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc9nr/gene_based/*feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

# motif databases
f_motif_path = "/ddn1/vol1/staging/leuven/res_00001/databases/cistarget/motif2tf/motifs-v9-nr.hgnc-m0.001-o0.0.tbl"

In [ ]:
!pyscenic ctx adj.tsv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 20

In [ ]:
nGenesDetectedPerCell = np.sum(adata.X>0, axis=1)
percentiles = nGenesDetectedPerCell.quantile([.01, .05, .10, .50, 1])
print(percentiles)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=150)
sns.distplot(nGenesDetectedPerCell, norm_hist=False, kde=False, bins='fd')
for i,x in enumerate(percentiles):
    fig.gca().axvline(x=x, ymin=0,ymax=1, color='red')
    ax.text(x=x, y=ax.get_ylim()[1], s=f'{int(x)} ({percentiles.index.values[i]*100}%)', color='red', rotation=30, size='x-small',rotation_mode='anchor' )
ax.set_xlabel('# of genes')
ax.set_ylabel('# of cells')
fig.tight_layout()

In [ ]:
!pyscenic aucell \
    {f_loom_path_scenic} \
    reg.csv \
    --output {f_pyscenic_output} \
    --num_workers 20

##### STEP 3: Cluster cells around regulons

In [ ]:
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()

In [ ]:
runUmap = umap.UMAP(n_neighbors=10, min_dist=0.4, metric='correlation').fit_transform
dr_umap = runUmap( auc_mtx )
pd.DataFrame(dr_umap, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_umap.txt", sep='\t')
# tSNE
tsne = TSNE( n_jobs=20 )
dr_tsne = tsne.fit_transform( auc_mtx )
pd.DataFrame(dr_tsne, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_tsne.txt", sep='\t')

#### STEP 4 Integrate the Output --- THIS LOOKS PROBLEMATIC AND VERY NOT RIGHT

In [ ]:
# scenic output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
meta = json.loads(zlib.decompress(base64.b64decode( lf.attrs.MetaData )))
#exprMat = pd.DataFrame( lf[:,:], index=lf.ra.Gene, columns=lf.ca.CellID)
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
regulons = lf.ra.Regulons
dr_umap = pd.read_csv( 'scenic_umap.txt', sep='\t', header=0, index_col=0 )
dr_tsne = pd.read_csv( 'scenic_tsne.txt', sep='\t', header=0, index_col=0 )
###

In [ ]:
auc_mtx.columns = auc_mtx.columns.str.replace('\(','_(')
regulons.dtype.names = tuple( [ x.replace("(","_(") for x in regulons.dtype.names ] )
# regulon thresholds
rt = meta['regulonThresholds']
for i,x in enumerate(rt):
    tmp = x.get('regulon').replace("(","_(")
    x.update( {'regulon': tmp} )

In [ ]:
tsneDF = pd.DataFrame(adata.obsm['X_tsne'], columns=['_X', '_Y'])

Embeddings_X = pd.DataFrame( index=lf.ca.CellID )
Embeddings_X = pd.concat( [
        pd.DataFrame(adata.obsm['X_umap'],index=adata.obs.index)[0] ,
        pd.DataFrame(adata.obsm['X_pca'],index=adata.obs.index)[0] ,
        dr_tsne['X'] ,
        dr_umap['X']
    ], sort=False, axis=1, join='outer' )
Embeddings_X.columns = ['1','2','3','4']

Embeddings_Y = pd.DataFrame( index=lf.ca.CellID )
Embeddings_Y = pd.concat( [
        pd.DataFrame(adata.obsm['X_umap'],index=adata.obs.index)[1] ,
        pd.DataFrame(adata.obsm['X_pca'],index=adata.obs.index)[1] ,
        dr_tsne['Y'] ,
        dr_umap['Y']
    ], sort=False, axis=1, join='outer' )
Embeddings_Y.columns = ['1','2','3','4']